In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

def imgshow(img):
    img_RGB = cv2.cvtColor(img,cv2.COLOR_BAYER_BG2RGB)
    plt.imshow(img_RGB)
    plt.show()

In [2]:
#1，导入预训练模型
net = cv2.dnn.readNet('yolov3.weights','yolov3.cfg')

error: OpenCV(4.5.1) C:\Users\appveyor\AppData\Local\Temp\1\pip-req-build-wvn_it83\opencv\modules\dnn\src\darknet\darknet_importer.cpp:214: error: (-212:Parsing error) Failed to parse NetParameter file: yolov3.weights in function 'cv::dnn::dnn4_v20201117::readNetFromDarknet'


In [ ]:
#2，导入COCO数据集的80个类别
with open('coco.names','r') as f:
    classes = f.read().splitlines()

In [ ]:
def predict_yolov3(img,classes,net):
    #3，预处理读入的图片
    #img = cv2.imread('test.png')
    h,w,_ = img.shape
    blob = cv2.dnn.blobFromImage(img,1/255,(416,416),(0,0,0),swapRB=True,crop=False)  #预处理图片

    net.setInput(blob)

    #前向推断
    layersNames = net.getLayerNames()
    output_layers_names = [layersNames[i[0]-1] for i in net.getUnconnectedOutLayers()]
    #output_layers_names = [yolo_82,yolo_94,yolo_106] 代表三个输出层，输出三组52*52*255，26*26*255，13*13*255
    prediction = net.forward(output_layers_names)
    #prediction是长度为3的列表，[507*85,2028*85,8112*85]，即13*13，26*26，52*52
    #4，backbone输出的prediction做neck和head处理
    boxes = []      #存放预测框坐标
    objectness = [] #存放置信度
    class_probs = []#存放类别概率
    class_ids = []  #存放预测框类别索引号
    class_names = []#存放预测框类别名称

    for scale in prediction:
        for bbox in scale:
            obj = bbox[4]
            class_scores = bbox[5:]
            class_id = np.argmax(class_scores)
            class_name = classes[class_id]
            class_prob = class_scores[class_id]
            #预测框的几何信息
            center_x = int(bbox[0]*w)
            center_y=  int(bbox[1]*h)
            real_w = int(bbox[2]*w) 
            real_h = int(bbox[3]*h)
            #左上角
            x = int(center_x - w/2)
            y = int(center_y - h/2)

            boxes.append([x,y,w,h])
            objectness.append(float(obj))
            class_ids.append(class_id)
            class_names.append(class_name)
            class_probs.append(class_prob)
    #boxes=13*13+26*26+52*52=10647

    confidences = np.array(class_probs) * np.array(objectness)# 计算绝对概率
    #5，NMS
    CONF_THRES = 0.1
    NMS_THRES = 0.4
    indexes = cv2.dnn.NMSBoxes(boxes,confidences,CONF_THRES,NMS_THRES)

    #6，遍历所有预测框+可视化
    colors = [[255,0,255],[0,0,255],[0,255,255],[0,255,0],[255,255,0],[255,0,0],[180,187,28],[223,155,6],[94,218,121]]
    for i in indexes.flatten():
        x,y,w,h = boxes[i]
        confidence = str(round(confidences[i],2))
        color = colors[i%len(colors)]
        cv2.rectangle(img,(x,y),(x+w,y+h),color,8)
        string = '{}:{}'.format(class_names[i],confidence)
        cv2.putText(img,string,(x,y+20),cv2.FONT_HERSHEY_PLAIN,3,(255,255,255),5)
    return img

In [ ]:
import cv2
cap = cv2.VideoCapture(1)
while True:
    grabbed, frame = cap.read()
    while not grabbed:
        continue
    cv2.imshow("capture", predict_yolov3(frame,classes,net))
    # cv2.waitKey(0)  # 捕获并显示一帧，按键后捕获并显示新的一帧
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()